1. Upload `.env` file into project directory `/home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments/cloud/sagemaker`.
2. Execute `setup.sh` in the same directory.
3. Then you can run this notebook

In [19]:
%pip install -r /home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments/requirements.txt --ignore-installed

Ignoring pywin32: markers 'python_full_version >= "3.9.16" and python_version < "3.12" and sys_platform == "win32"' don't match your environment
Ignoring waitress: markers 'python_full_version >= "3.9.16" and python_version < "3.12" and platform_system == "Windows"' don't match your environment
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached aiobotocore-2.5.4-py3-none-any.whl (73 kB)
  Using cached aiohttp_retry-2.8.3-py3-none-any.whl (9.8 kB)
  Using cached aiohttp-3.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached aioitertools-0.11.0-py3-none-any.whl (23 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached alembic-1.12.0-py3-none-any.whl (226 kB)
  Using cached amqp-5.1.1-py3-none-any.whl (50 kB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached async_timeout-4.

In [6]:
%cd /home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments

/home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments


In [ ]:
!python data.py

In [ ]:
# if you want to remove checkpoints before starting the training
rm -rf model/checkpoints

In [16]:
!python train.py

2023-09-23 11:16:36.838926: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 11:16:36.891770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 11:16:36.892363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 11:16:37.853565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023/09/23 11:16:40 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(No

Quick evaluation

In [17]:
path = !(echo $(pwd))

In [20]:
import sys
sys.path.insert(0, path[0])

import numpy as np
import chess
import tensorflow
from tensorflow import keras

from utils.board_representation import tensor_to_board
from utils.visualisation import binarize_array, print_board
from utils.data_loader import load_train_test
from train import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss

2023-09-23 12:54:02.873688: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 12:54:02.925930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 12:54:02.927033: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 12:54:03.870049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:
train_data, test_data = load_train_test("./data", "position_tensor_000")
print(len(train_data), len(test_data))

900000 100000


In [22]:
model = keras.models.load_model(
    "./model/20230923122501_autoencoder.keras",
    custom_objects={
        "sum_squared_loss": sum_squared_loss,
        "num_pc_reg": num_pc_reg,
        "pc_column_reg": pc_column_reg,
        "pc_plane_reg": pc_plane_reg,
        "custom_regularized_loss": custom_regularized_loss
    }
)

In [25]:
indices = [i for i in range(10)]
for index in indices:
    print(f"Compare boards for index {index}")
    sample = test_data[index].reshape((1,8,8,15))
    reconstruction = model.predict_on_batch(sample)
    print(model.evaluate(x=[reconstruction], y=[sample]))
    print(custom_regularized_loss(sample, reconstruction), sum_squared_loss(sample, reconstruction), num_pc_reg(sample, reconstruction), pc_column_reg(sample, reconstruction), pc_plane_reg(sample, reconstruction))
    binary_reconstruction = binarize_array(reconstruction)
    sample_board = tensor_to_board(sample[0])
    reconstructed_board = tensor_to_board(binary_reconstruction[0])
    print_board(sample_board)
    print("\n")
    print_board(reconstructed_board)

Compare boards for index 0
1/1 [==============================] - 0s 26ms/step - loss: 1.5938 - sum_squared_loss: 1.3438 - num_pc_reg: 4.3678e-04 - pc_column_reg: 1.2578 - pc_plane_reg: 1.2422
[1.59375, 1.34375, 0.0004367828369140625, 1.2578125, 1.2421875]
tf.Tensor(1.26562, shape=(), dtype=bfloat16) tf.Tensor(1.08594, shape=(), dtype=bfloat16) tf.Tensor(0.0163574, shape=(), dtype=bfloat16) tf.Tensor(0.820312, shape=(), dtype=bfloat16) tf.Tensor(0.789062, shape=(), dtype=bfloat16)
. . N . . . . .
. . . . . . . .
. N . . . . k .
. . . . . . . .
. . . . n . . .
. . . . . . . .
B . . . . . . .
Q R K . . . . .




. . . . . . . .
. . . . . . . .
. N . . . . k .
. . . . . . . .
. . . . n . . .
. . . . . . . .
B . . . . . . .
Q R K . . . . .


Compare boards for index 1
1/1 [==============================] - 0s 23ms/step - loss: 0.0079 - sum_squared_loss: 0.0055 - num_pc_reg: 0.0013 - pc_column_reg: 0.0055 - pc_plane_reg: 0.0055
[0.00787353515625, 0.0054931640625, 0.00127410888671875, 0.0054

If that's promising then try to save artifacts

In [ ]:
!dvc add model

In [ ]:
!git add --all && git status

In [ ]:
!git commit -m "Colab model loss xy at batch yz"

In [ ]:
!git push origin master

In [ ]:
!dvc push -r colab_origin